# Consignes

projet : https://openclassrooms.com/fr/projects/630/assignment
données : https://www.kaggle.com/olistbr/brazilian-ecommerce

Olist souhaite que vous fournissiez à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Enfin, votre client, Olist, a spécifié sa demande ainsi :

* La segmentation proposée doit être exploitable et facile d’utilisation pour l’équipe marketing.
* Vous évaluerez la fréquence à laquelle la segmentation doit être mise à jour, afin de pouvoir effectuer un devis de contrat de maintenance.
* Le code fourni doit respecter la convention PEP8, pour être utilisable par Olist.

Livrables

* Un notebook de l'analyse exploratoire (non cleané, pour comprendre votre démarche).
* Un notebook (ou code commenté au choix) d’essais des différentes approches de modélisation (non cleané, pour comprendre votre démarche).
* Un support de présentation pour la soutenance.


In [ ]:
# %pip install geopandas
# !pip install plotly
# !pip install folium
# !pip install ipyleaflet
# !pip install bokeh


# Import et données 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import duckdb as ddb
import folium
from folium.plugins import MarkerCluster
# Utilisation de Geopandas (pour les données géospatiales) :
import geopandas as gpd
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", 101)
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')

In [ ]:
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocalisation = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
translation = pd.read_csv('data/product_category_name_translation.csv')

In [ ]:
# Définition d'une fonction pour tracer une heatmap
# paramètres :
# corr : matrice de corrélation
# title : titre du graphe
# figsize : taille de la figure
# vmin : valeur minimale de la colormap
# vmax : valeur maximale de la colormap
# center : valeur centrale de la colormap
# palette : palette de couleurs
# shape : forme de la heatmap (rectangle ou triangle)
# fmt : format des nombres affichés
# robust : booléen pour utiliser une méthode de calcul robuste ou non
def plot_heatmap(corr, title, figsize=(8,4), vmin=-1, vmax=1, center=0,
                 palette = sns.color_palette("coolwarm", 20), shape='rect',
                 fmt='.2f', robust=False):
    
    # Création d'une figure et d'un axe
    fig, ax = plt.subplots(figsize=figsize)
    
    # Définition du masque pour la forme de la heatmap
    if shape == 'rect':
        mask=None
    elif shape == 'tri':
        mask = np.zeros_like(corr, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
    else:
        print('ERROR : this type of heatmap does not exist')

    # Définition de la palette de couleurs
    palette = palette
    
    # Tracé de la heatmap
    ax = sns.heatmap(corr, mask=mask, cmap=palette, vmin=vmin, vmax=vmax,
                     center=center, annot=True, annot_kws={"size": 10},fmt=fmt,
                     square=False, linewidths=.5, linecolor = 'white',
                     cbar_kws={"shrink": .9, 'label': None}, robust = robust,
                     xticklabels= corr.columns, yticklabels = corr.index)
    
    # Configuration des axes et de la légende
    ax.tick_params(labelsize=8,top=False, bottom=True,
                labeltop=False, labelbottom=True)
    ax.collections[0].colorbar.ax.tick_params(labelsize=8)
    plt.setp(ax.get_xticklabels(), rotation=25, ha="right",rotation_mode="anchor")
    ax.set_title(title, fontweight='bold', fontsize=14)
    
    # Retourne le masque (inutilisé dans ce code)
    # return mask


In [ ]:
# def show_correlogram(data):
#     numeric_columns = data.select(cs.numeric())
#     corr = numeric_columns.to_pandas().corr()
#     mask = np.triu(corr)
#     sns.set(rc={"figure.figsize": (20, 10)})
#     sns.heatmap(corr, annot=True, fmt='.2f', 
#                 # mask=mask,
#                 vmin=-1, vmax=1, center=0, cmap='coolwarm')
def show_correlogram(data):
    numeric_columns = data.select_dtypes(include='number')
    corr = numeric_columns.corr()
    mask = np.triu(corr)
    sns.set(rc={"figure.figsize": (20, 10)})
#     sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", mask=mask)
    sns.heatmap(corr, annot=True, fmt='.2f', 
                # mask=mask,
                vmin=-1, vmax=1, center=0, cmap='coolwarm')
    plt.title("Corrélogramme des colonnes numériques")
    plt.show()
    

# Connaissance du jeu de données et Nettoyage

## Observation 

### Contenu 

In [ ]:
customers.head(3)

In [ ]:
geolocalisation.head(2)

In [ ]:
order_items.head(2)

In [ ]:
order_payments.head(2)

In [ ]:
order_reviews.head(2)

In [ ]:
orders.head(2)

In [ ]:
products.head(2)

In [ ]:
sellers.head(2)

In [ ]:
translation.head(2)

1. Fichier "customers.csv" :
   - Résumé : Contient des informations sur les clients qui ont effectué des achats auprès d'Olist.
   - Colonnes importantes :
     - customer_id : Identifiant unique du client.
     - customer_unique_id : Identifiant unique du client, utilisé pour relier les informations du client à ses commandes.
     - customer_zip_code_prefix : Code postal du client.
     - customer_city : Ville du client.
     - customer_state : État du client.

2. Fichier "geolocation.csv" :
   - Résumé : Contient les informations de géolocalisation des différents codes postaux du Brésil.
   - Colonnes importantes :
     - geolocation_zip_code_prefix : Code postal.
     - geolocation_lat : Latitude géographique.
     - geolocation_lng : Longitude géographique.
     - geolocation_city : Ville correspondante au code postal.
     - geolocation_state : État correspondant au code postal.

3. Fichier "order_items.csv" :
   - Résumé : Contient des informations détaillées sur les articles inclus dans chaque commande.
   - Colonnes importantes :
     - order_id : Identifiant unique de la commande.
     - product_id : Identifiant unique du produit.
     - seller_id : Identifiant unique du vendeur.
     - shipping_limit_date : Date limite d'expédition du produit.

4. Fichier "order_payments.csv" :
   - Résumé : Contient les informations sur les paiements effectués pour chaque commande.
   - Colonnes importantes :
     - order_id : Identifiant unique de la commande.
     - payment_sequential : Numéro de séquence du paiement pour une commande donnée.
     - payment_type : Type de paiement utilisé (carte de crédit, virement bancaire, etc.).
     - payment_installments : Nombre d'installments (versements) pour le paiement.
     - payment_value : Montant total du paiement.

5. Fichier "order_reviews.csv" :
   - Résumé : Contient les avis et évaluations laissés par les clients pour chaque commande.
   - Colonnes importantes :
     - review_id : Identifiant unique de l'avis.
     - order_id : Identifiant unique de la commande associée à l'avis.
     - review_score : Note donnée par le client (de 1 à 5).
     - review_comment_title : Titre du commentaire laissé par le client.
     - review_comment_message : Contenu du commentaire laissé par le client.

6. Fichier "orders.csv" :
   - Résumé : Contient des informations générales sur les commandes passées par les clients.
   - Colonnes importantes :
     - order_id : Identifiant unique de la commande.
     - customer_id : Identifiant unique du client.
     - order_status : Statut de la commande (livrée, en cours de traitement, etc.).
     - order_purchase_timestamp : Date et heure d'achat de la commande.
     - order_approved_at : Date et heure d'approbation de la commande.
     - order_delivered_customer_date : Date de livraison estimée au client.
     - order_estimated_delivery_date : Date estimée de livraison.

7. Fichier "products.csv" :
   - Résumé : Contient des informations sur les produits vendus par Olist.
   - Colonnes importantes :
     - product_id : Identifiant unique du produit.
     - product_category_name : Nom de la catégorie de produit.
     - product_name_length : Longueur du nom du produit.
     - product_description_length : Longueur de la description du produit.

8. Fichier "sellers.csv" :
   - Résumé : Contient des informations sur les vendeurs partenaires d'Olist.
   - Colonnes importantes :
     - seller_id : Identifiant unique du vendeur.
     - seller_zip_code_prefix : Code postal du vendeur.
     - seller_city : Ville du vendeur.
     - seller_state : État du vendeur.

9. Fichier "product_category_name_translation.csv" :
   - Résumé : Contient les traductions des noms de catégories de produits en différentes langues.
   - Colonnes importantes :
     - product_category_name : Nom de la catégorie de produit en portugais (à traduire en anglais).


In [ ]:
liste_df = [customers, 
            geolocalisation,
            order_items,
            order_payments,
            order_reviews,
            orders,products,
            sellers,
            translation]

In [ ]:
noms_df = ['customers', 
           'geolocalisation',
           'order_items',
           'order_payments',
           'order_reviews',
           'orders',
           'products',
           'sellers',
           'translation']

### Dimensions des jeux de données

In [ ]:
for df in liste_df :
    print(df.shape)

In [ ]:
i = 0
for df in liste_df:
#     nom_dataframe = noms_df[df
    
    print(f"Le dataframe {noms_df[i]} a {df.shape[0]} lignes et {df.shape[1]} colonnes.")
    i = i + 1


### Données manquantes

In [ ]:
# for df in liste_df:
#     print(df.isna().sum().sum()/df.shape[0]/df.shape[1], 'NaN')
# Parcours des DataFrames dans la liste 'liste_df'
for df in liste_df:
    # Calcule le pourcentage de valeurs NaN dans chaque DataFrame
    # isna().sum() compte le nombre de valeurs manquantes dans chaque colonne
    # sum() somme le nombre de valeurs manquantes dans toutes les colonnes
    # df.shape[0] donne le nombre de lignes dans le DataFrame
    # df.shape[1] donne le nombre de colonnes dans le DataFrame
    # On divise le nombre total de valeurs manquantes par le nombre total de valeurs dans le DataFrame
    # pour obtenir le pourcentage de valeurs NaN dans le DataFrame
    pourcentage_nan = df.isna().sum().sum() / (df.shape[0] * df.shape[1])
    
    # Affiche le pourcentage de valeurs NaN dans le DataFrame
    print(pourcentage_nan, 'NaN')

Ce code parcourt chaque DataFrame dans la liste liste_df et pour chaque DataFrame, il calcule le pourcentage de valeurs manquantes (NaN) en comptant le nombre total de valeurs manquantes dans toutes les colonnes et en le divisant par le nombre total de valeurs dans le DataFrame (nombre de lignes multiplié par le nombre de colonnes). Ensuite, il affiche le pourcentage de valeurs NaN pour chaque DataFrame.

In [ ]:
# # Parcours des DataFrames dans la liste 'liste_df'
# for df in liste_df:
#     # Calcule le pourcentage de valeurs NaN dans chaque DataFrame
#     # isna().sum() compte le nombre de valeurs manquantes dans chaque colonne
#     # sum() somme le nombre de valeurs manquantes dans toutes les colonnes
#     # df.shape[0] donne le nombre de lignes dans le DataFrame
#     # df.shape[1] donne le nombre de colonnes dans le DataFrame
#     # On divise le nombre total de valeurs manquantes par le nombre total de valeurs dans le DataFrame
#     # pour obtenir le pourcentage de valeurs NaN dans le DataFrame
#     pourcentage_nan = df.isna().sum().sum() / (df.shape[0] * df.shape[1])
    
#     # Affiche le pourcentage de valeurs NaN dans le DataFrame avec le libellé du DataFrame
#     print(f"Pourcentage de valeurs NaN dans le DataFrame '{df}' : {pourcentage_nan:.2%}")


On a des NaN uniquement des order_reviews, orders et products

In [ ]:
# order_reviews.isna().sum(axis=0)
# Calculer le nombre de valeurs manquantes dans chaque colonne du DataFrame 'order_reviews'
# La méthode isna() retourne un DataFrame avec des valeurs booléennes (True pour les valeurs manquantes, False sinon)
# La méthode sum() retourne la somme des valeurs booléennes pour chaque colonne (True est équivalent à 1, False est équivalent à 0)
# L'argument axis=0 spécifie que la somme doit être effectuée par colonne
# Ainsi, le résultat sera le nombre de valeurs manquantes dans chaque colonne du DataFrame 'order_reviews'
order_reviews_missing_values = order_reviews.isna().sum(axis=0)

# Afficher le nombre de valeurs manquantes dans chaque colonne du DataFrame 'order_reviews'
print(order_reviews_missing_values)


Ce code calcule le nombre de valeurs manquantes dans chaque colonne du DataFrame order_reviews en utilisant la méthode isna() pour identifier les valeurs manquantes et la méthode sum() pour compter le nombre de valeurs manquantes dans chaque colonne. Ensuite, il affiche le nombre de valeurs manquantes pour chaque colonne du DataFrame order_reviews.

pour order_reviews, les NaN correspondent à  des commentaires vides

In [ ]:
# orders.isna().sum(axis=0)
# Calculer le nombre de valeurs manquantes dans chaque colonne du DataFrame 'orders'
# La méthode isna() retourne un DataFrame avec des valeurs booléennes (True pour les valeurs manquantes, False sinon)
# La méthode sum() retourne la somme des valeurs booléennes pour chaque colonne (True est équivalent à 1, False est équivalent à 0)
# L'argument axis=0 spécifie que la somme doit être effectuée par colonne
# Ainsi, le résultat sera le nombre de valeurs manquantes dans chaque colonne du DataFrame 'orders'
orders_missing_values = orders.isna().sum(axis=0)

# Afficher le nombre de valeurs manquantes dans chaque colonne du DataFrame 'orders'
print(orders_missing_values)

Ce code calcule le nombre de valeurs manquantes dans chaque colonne du DataFrame orders en utilisant la méthode isna() pour identifier les valeurs manquantes et la méthode sum() pour compter le nombre de valeurs manquantes dans chaque colonne. Ensuite, il affiche le nombre de valeurs manquantes pour chaque colonne du DataFrame orders.

Pour les commandes, les NaN sont croissants dans l'ordre d'un processus de commande et correspondent aux commandes qui posent problème

In [ ]:
# orders.max(
# Trouver la valeur maximale dans chaque colonne du DataFrame 'orders'
# La méthode max() retourne la valeur maximale dans chaque colonne du DataFrame
# Si les colonnes contiennent des valeurs numériques, la méthode max() renverra la valeur numérique maximale
# Si les colonnes contiennent des valeurs non numériques, la méthode max() renverra la valeur maximale dans l'ordre lexicographique
orders_max_values = orders.max()

# Afficher la valeur maximale dans chaque colonne du DataFrame 'orders'
print(orders_max_values)

Ce code trouve la valeur maximale dans chaque colonne du DataFrame orders en utilisant la méthode max(). Si les colonnes contiennent des valeurs numériques, la méthode max() renverra la valeur numérique maximale dans chaque colonne. Si les colonnes contiennent des valeurs non numériques, la méthode max() renverra la valeur maximale dans l'ordre lexicographique. Ensuite, il affiche la valeur maximale dans chaque colonne du DataFrame orders.

In [ ]:
orders.order_approved_at

In [ ]:
# products.isna().sum(axis=0)
# Calculer le nombre de valeurs manquantes dans chaque colonne du DataFrame 'products'
# La méthode isna() retourne un DataFrame avec des valeurs booléennes (True pour les valeurs manquantes, False sinon)
# La méthode sum() retourne la somme des valeurs booléennes pour chaque colonne (True est équivalent à 1, False est équivalent à 0)
# L'argument axis=0 spécifie que la somme doit être effectuée par colonne
# Ainsi, le résultat sera le nombre de valeurs manquantes dans chaque colonne du DataFrame 'products'
products_missing_values = products.isna().sum(axis=0)

# Afficher le nombre de valeurs manquantes dans chaque colonne du DataFrame 'products'
print(products_missing_values)


In [ ]:
products.shape

In [ ]:
products[products['product_category_name'].isna()].sample(5)

On a des produits pour lesquels il manque des informations importantes (en particulier la catégorie de produit).

On pourra éventuellement leur donner des attributs correspondant à leur caractère inconnu

In [ ]:
# products[products['product_weight_g'].isna()]
# Sélectionner les lignes du DataFrame 'products' où la colonne 'product_weight_g' contient des valeurs manquantes (NaN)
# La condition products['product_weight_g'].isna() renvoie une série booléenne indiquant True pour les valeurs manquantes et False sinon
# En utilisant cette série booléenne comme masque, nous sélectionnons les lignes où la colonne 'product_weight_g' est manquante
products_missing_weight = products[products['product_weight_g'].isna()]

# Afficher les lignes du DataFrame 'products' où la colonne 'product_weight_g' est manquante
products_missing_weight

### doublons 

In [ ]:
# for df in liste_df:
#     print(df.duplicated().sum())
# Pour chaque DataFrame 'df' dans la liste 'liste_df'
for df in liste_df:
    # Calculer le nombre de lignes dupliquées dans le DataFrame 'df'
    # La méthode 'duplicated()' renvoie une série booléenne indiquant True pour les lignes dupliquées et False sinon
    # En utilisant la méthode 'sum()', nous comptons le nombre total de lignes dupliquées en sommant les valeurs True (1) dans la série
    nb_duplicates = df.duplicated().sum()

    # Afficher le nombre de lignes dupliquées dans le DataFrame 'df'
    print(nb_duplicates)


###  Types des données et mode des données

In [ ]:
# i=0
# liste_indices = []
# liste_colonnes = []
# liste_types = []
# liste_uniques = []

# for df in liste_df:

#     for column in df.columns:
#         liste_indices.append(noms_df[i])
#         liste_colonnes.append(column)
#         liste_types.append(df[column].dtype)
#         liste_uniques.append(df[column].nunique())
#     i+=1
# resume_data = pd.DataFrame([liste_indices, 
#                             liste_colonnes, 
#                             liste_types, 
#                             liste_uniques]
#                           ).T
# resume_data.columns=['Donnees', 'Nom colonne', 'Type', 'valeurs uniques']
# resume_data


In [ ]:
# Initialisation des listes qui contiendront les informations résumées
i = 0
liste_indices = []
liste_colonnes = []
liste_types = []
liste_uniques = []

# Parcours de chaque DataFrame 'df' dans la liste 'liste_df'
for df in liste_df:
    # Parcours de chaque colonne 'column' dans le DataFrame 'df'
    for column in df.columns:
        # Ajout des informations de chaque colonne dans les listes correspondantes
        liste_indices.append(noms_df[i])   # Nom du DataFrame 'df' correspondant (utilisation de noms_df[i])
        liste_colonnes.append(column)      # Nom de la colonne
        liste_types.append(df[column].dtype)  # Type de données de la colonne
        liste_uniques.append(df[column].nunique())  # Nombre de valeurs uniques dans la colonne

    i += 1  # Passage au DataFrame suivant

# Création d'un DataFrame résumant les informations collectées
resume_data = pd.DataFrame([liste_indices, liste_colonnes, liste_types, liste_uniques]).T
resume_data.columns = ['Donnees', 'Nom colonne', 'Type', 'Valeurs uniques']

# Affichage du DataFrame résumé
resume_data


Ce code parcourt chaque DataFrame de la liste liste_df, puis chaque colonne de chaque DataFrame, et collecte les informations telles que le nom du DataFrame correspondant, le nom de la colonne, le type de données de la colonne et le nombre de valeurs uniques dans la colonne. Ces informations sont ensuite rassemblées dans un DataFrame résumé appelé resume_data, qui est ensuite affiché.

Observations:
* customers
     * davantage de valeurs différentes pour customer_id que customer_unique_id. 
     * customer_state pourrait être catéogrisé
* geolocalisation
    * plus de villes que dans la base de données customers :
    * geolocation_state pourrait être catégorisé
* order_items
    * order_item_id contient 21 types différents. 
    * autant de seller_id que dans la base ed données sellers
    * shipping_limit_date : à mettre en format date
* order_payments :
    * order_id contient autant de valeurs que dans order
    * payment_sequential et payment_type pourraient être catégories
* orders 
    * order_purchase_timestamp : à mettre en format date
    * order_approved_at :  à mettre en format date
    * order_delivered_carrier_date :  à mettre en format date
    * order_delivered_customer_date :  à mettre en format date
    * order_estimated_delivery_date :  à mettre en format date
* products:
    * 73 catégories différentes
* sellers:
* translation : 3 catégories non traduites

## Fonction de nettoyage 

### contrôle

In [ ]:
# def controle_df(liste_donnees):
#     '''Vérifications de type et de la taille du jeu de données pour correspondre 
#     au jeu initial
    
#     Prend en entrée une liste de dataframes
#     Retourne un Booléen
#     True dans le cas d'une liste de 9 objects de type dataframe
#     False dans le cas contraire
#     '''
#     if type(liste_donnees) is list:
#         if len(liste_donnees) == 9:
#             for df in liste_donnees:
#                 if type(df) is not type(pd.DataFrame()):
#                     return False
#         else:
#             return False
#     else:
#         return False
#     return True
def controle_df(liste_donnees):
    '''
    Vérifications de type et de la taille du jeu de données pour correspondre 
    au jeu initial
    
    Prend en entrée une liste de dataframes
    Retourne un Booléen
    True dans le cas d'une liste de 9 objets de type dataframe
    False dans le cas contraire
    '''
    # Vérifie si la variable 'liste_donnees' est de type liste
    if type(liste_donnees) is list:
        # Vérifie si la liste contient 9 éléments (dataframes)
        if len(liste_donnees) == 9:
            # Parcours chaque dataframe 'df' dans la liste 'liste_donnees'
            for df in liste_donnees:
                # Vérifie si chaque élément de la liste est bien un objet de type dataframe
                if type(df) is not type(pd.DataFrame()):
                    # Si ce n'est pas le cas, renvoie False (la liste ne contient pas 9 dataframes)
                    return False
        else:
            # Si la liste ne contient pas 9 éléments, renvoie False
            return False
    else:
        # Si 'liste_donnees' n'est pas de type liste, renvoie False
        return False
    
    # Si toutes les vérifications sont passées, renvoie True (la liste contient bien 9 dataframes)
    return True


Cette fonction controle_df prend en entrée une liste de dataframes, et elle vérifie si la liste contient bien 9 objets de type dataframe. Si c'est le cas, elle renvoie True, sinon elle renvoie False.

### Nettoyage

In [ ]:
# def nettoyage(liste_donnees):
#     '''Nettoyage des différents dataframe
#     Entrée : liste de 9 dataframes
    
#     Traitement appliqué : 
#     * complétion des NaN, 
#     * nettoyage type, 
#     * merge catégories de produits
    
#     Return : liste des dataframe nettoyée
#     '''
#     if not controle_df(liste_donnees):
#         return False

#     customers_local = liste_donnees[0].copy()
#     geolocalisation_local  = liste_donnees[1].copy()
#     order_items_local  = liste_donnees[2].copy()
#     order_payments_local  = liste_donnees[3].copy()
#     order_reviews_local  = liste_donnees[4].copy()
#     orders_local  = liste_donnees[5].copy()
#     products_local = liste_donnees[6].copy()
#     sellers_local = liste_donnees[7].copy()
#     translation_local = liste_donnees[8].copy()

    
#     #Traitement des NaN
#     geolocalisation_local.drop_duplicates(inplace=True)
    
#     order_reviews_local['review_comment_title'].fillna(' ',
#                                                        inplace=True)
#     order_reviews_local['review_comment_message'].fillna(' ',
#                                                          inplace=True)
    
#     orders['order_approved_at'].fillna(-1, inplace=True)
#     orders['order_delivered_carrier_date'].fillna(0, inplace=True)
#     orders['order_delivered_customer_date'].fillna(0, inplace=True)
    
#     products_local['product_category_name'].fillna('Unkwown', 
#                                                    inplace=True)
#     products_local['product_name_lenght'].fillna(0, inplace=True)
#     products_local['product_description_lenght'].fillna(0, 
#                                                         inplace=True)
#     products_local['product_photos_qty'].fillna(0, inplace=True)
#     products_local['product_weight_g'].fillna(0, inplace=True)
#     products_local['product_length_cm'].fillna(0, inplace=True)
#     products_local['product_height_cm'].fillna(0, inplace=True)
#     products_local['product_width_cm'].fillna(0, inplace=True)
    
#     #nettoyage types:
#     order_items_local['shipping_limit_date'] = order_items_local[
#         'shipping_limit_date'].astype('datetime64')
#     orders_local['order_purchase_timestamp'] = orders_local[
#         'order_purchase_timestamp'].astype('datetime64')
#     orders_local['order_approved_at'] = pd.to_datetime(orders_local[
#         'order_approved_at'], errors='coerce')
#     orders_local['order_delivered_carrier_date'] = pd.to_datetime(
#         orders_local['order_delivered_carrier_date'], errors='coerce')
#     orders_local['order_delivered_customer_date'] = pd.to_datetime(
#         orders_local['order_delivered_customer_date'], errors='coerce')
#     orders_local['order_estimated_delivery_date'] = orders_local[
#         'order_estimated_delivery_date'].astype('datetime64')
    
#     #merge des catégories de produits
#     products_local = pd.merge(products_local, translation_local).drop(
#         ['product_category_name'], axis=1)
    
#     return [
#         customers_local, 
#         geolocalisation_local, 
#         order_items_local, 
#         order_payments_local, 
#         order_reviews_local, 
#         orders_local, 
#         products_local, 
#         sellers_local, 
#         translation_local]

In [ ]:
def nettoyage(liste_donnees):
    '''
    Nettoyage des différents dataframes
    Entrée : liste de 9 dataframes
    
    Traitement appliqué : 
    * complétion des NaN, 
    * nettoyage des types de données, 
    * fusion des catégories de produits
    
    Retourne : liste des dataframes nettoyés
    '''
    # Vérifie si la liste de données est valide en appelant la fonction controle_df
    if not controle_df(liste_donnees):
        return False

    # Effectue une copie des dataframes pour éviter de modifier les dataframes d'origine
    customers_local = liste_donnees[0].copy()
    geolocalisation_local  = liste_donnees[1].copy()
    order_items_local  = liste_donnees[2].copy()
    order_payments_local  = liste_donnees[3].copy()
    order_reviews_local  = liste_donnees[4].copy()
    orders_local  = liste_donnees[5].copy()
    products_local = liste_donnees[6].copy()
    sellers_local = liste_donnees[7].copy()
    translation_local = liste_donnees[8].copy()

    # Traitement des NaN

    # Supprime les lignes dupliquées dans le dataframe geolocalisation_local
    geolocalisation_local.drop_duplicates(inplace=True)

    # Remplace les valeurs manquantes dans les colonnes 'review_comment_title' et 'review_comment_message' par des espaces vides
    order_reviews_local['review_comment_title'].fillna(' ', inplace=True)
    order_reviews_local['review_comment_message'].fillna(' ', inplace=True)

    # Remplace les valeurs manquantes dans certaines colonnes du dataframe orders_local
    orders['order_approved_at'].fillna(-1, inplace=True)
    orders['order_delivered_carrier_date'].fillna(0, inplace=True)
    orders['order_delivered_customer_date'].fillna(0, inplace=True)

    # Remplace les valeurs manquantes dans certaines colonnes du dataframe products_local par des valeurs par défaut
    products_local['product_category_name'].fillna('Unkwown', inplace=True)
    products_local['product_name_lenght'].fillna(0, inplace=True)
    products_local['product_description_lenght'].fillna(0, inplace=True)
    products_local['product_photos_qty'].fillna(0, inplace=True)
    products_local['product_weight_g'].fillna(0, inplace=True)
    products_local['product_length_cm'].fillna(0, inplace=True)
    products_local['product_height_cm'].fillna(0, inplace=True)
    products_local['product_width_cm'].fillna(0, inplace=True)

    # Nettoyage des types de données:

    # Convertit la colonne 'shipping_limit_date' du dataframe order_items_local en type datetime64
    order_items_local['shipping_limit_date'] = order_items_local['shipping_limit_date'].astype('datetime64')

    # Convertit certaines colonnes du dataframe orders_local en type datetime64
    orders_local['order_purchase_timestamp'] = orders_local['order_purchase_timestamp'].astype('datetime64')
    orders_local['order_approved_at'] = pd.to_datetime(orders_local['order_approved_at'], errors='coerce')
    orders_local['order_delivered_carrier_date'] = pd.to_datetime(orders_local['order_delivered_carrier_date'], errors='coerce')
    orders_local['order_delivered_customer_date'] = pd.to_datetime(orders_local['order_delivered_customer_date'], errors='coerce')
    orders_local['order_estimated_delivery_date'] = orders_local['order_estimated_delivery_date'].astype('datetime64')

    # Fusion des catégories de produits

    # Fusionne les dataframes products_local et translation_local en utilisant la colonne 'product_category_name' comme clé de fusion
    # Supprime ensuite la colonne 'product_category_name' du dataframe products_local
    products_local = pd.merge(products_local, translation_local).drop(['product_category_name'], axis=1)

    # Retourne la liste des dataframes nettoyés
    return [customers_local, 
            geolocalisation_local, 
            order_items_local, 
            order_payments_local, 
            order_reviews_local, 
            orders_local, 
            products_local, 
            sellers_local, 
            translation_local]


Cette fonction `nettoyage` prend en entrée une liste de 9 dataframes et effectue les opérations de nettoyage décrites dans les commentaires. Elle renvoie ensuite une liste contenant les dataframes nettoyés.

In [ ]:
liste_df = nettoyage(liste_df)

In [ ]:
liste_df

### Assemblage, traitement outliers et feature engineering

mapping categories

In [ ]:
# Définition d'un dictionnaire pour regrouper les catégories de produits en catégories générales

dict_categories = {
    # Catégorie : 'home'
    'furniture_living_room': 'home',
    'furniture_mattress_and_upholstery': 'home',
    'furniture_bedroom': 'home',
    'furniture_decor': 'home',
    'bed_bath_table': 'home',
    'kitchen_dining_laundry_garden_furniture': 'home',
    'la_cuisine': 'home',
    'home_confort': 'home',
    'home_comfort_2': 'home',
    'christmas_supplies': 'home',

    # Catégorie : 'appliances' = appareils électroménagers
    'small_appliances': 'appliances',
    'small_appliances_home_oven_and_coffee': 'appliances',
    'home_appliances_2': 'appliances',
    'home_appliances': 'appliances',
    'housewares': 'appliances',

    # Catégorie : 'construction'
    'construction_tools_construction': 'construction',
    'costruction_tools_garden': 'construction',
    'costruction_tools_tools': 'construction',
    'construction_tools_safety': 'construction',
    'construction_tools_lights': 'construction',
    'home_construction': 'construction',
    'air_conditioning': 'construction',

    # Catégorie : 'office'
    'office_furniture': 'office',
    'industry_commerce_and_business': 'office',
    'stationery': 'office',
    'agro_industry_and_commerce': 'office',
    'signaling_and_security': 'office',
    'furnitures': 'office',
    'security_and_services': 'office',

    # Catégorie : 'electronics'
    'telephony': 'electronics',
    'electronics': 'electronics',
    'computers_accessories': 'electronics',
    'consoles_games': 'electronics',
    'fixed_telephony': 'electronics',
    'audio': 'electronics',
    'computers': 'electronics',
    'tablets_printing_image': 'electronics',

    # Catégorie : 'sports_leisure'
    'sports_leisure': 'sports_leisure',
    'musical_instruments': 'sports_leisure',
    'party_supplies': 'sports_leisure',
    'luggage_accessories': 'sports_leisure',

    # Catégorie : 'arts'
    'books': 'arts',
    'books_imported': 'arts',
    'books_general_interest': 'arts',
    'books_technical': 'arts',
    'art': 'arts',
    'toys': 'arts',
    'cine_photo': 'arts',
    'cds_dvds_musicals': 'arts',
    'music': 'arts',
    'dvds_blu_ray': 'arts',
    'arts_and_craftmanship': 'arts',

    # Catégorie : 'fashion'
    'watches_gifts': 'fashion',
    'fashion_bags_accessories': 'fashion',
    'fashion_underwear_beach': 'fashion',
    'fashion_shoes': 'fashion',
    'fashion_male_clothing': 'fashion',
    'fashio_female_clothing': 'fashion',
    'fashion_sport': 'fashion',
    'fashion_childrens_clothes': 'fashion',

    # Catégorie : 'health_beauty'
    'health_beauty': 'health_beauty',
    'baby': 'health_beauty',
    'diapers_and_hygiene': 'health_beauty',
    'perfumery': 'health_beauty',

    # Catégorie : 'garden_pets'
    'flowers': 'garden_pets',
    'pet_shop': 'garden_pets',
    'garden_tools': 'garden_pets',

    # Catégorie : 'auto'
    'auto': 'auto',

    # Catégorie : 'food_drinks'
    'food_drink': 'food_drinks',
    'food': 'food_drinks',
    'drinks': 'food_drinks',

    # Catégorie : 'other'
    'Unknown': 'other',
    'market_place': 'other',
    'cool_stuff': 'other'
}


Ce dictionnaire dict_categories regroupe les catégories de produits en catégories plus générales. Chaque clé du dictionnaire représente une catégorie de produits spécifique, tandis que les valeurs associées représentent les catégories générales dans lesquelles ces produits sont regroupés. Par exemple, tous les produits liés à la maison sont regroupés sous la catégorie 'home', et tous les produits liés aux appareils sont regroupés sous la catégorie 'appliances', et ainsi de suite. Cela permet de simplifier et de mieux organiser les données relatives aux produits.

outliers

In [ ]:
# from sklearn.neighbors import LocalOutlierFactor

# def delete_univariate_outliers(dataframe):
#     '''Suppression des valeurs extrêmes du dataset - on exclut le centile le plus extreme
#     Entree: objet dataframe
#     Traitement : Supression Nan univariés
#     Sortie : objet dataframe
#     '''
#     #valeurs extremes
#     index_nan = []
#     index_nan_flat = []
#     for column in dataframe.select_dtypes(include = ['int32','float64']).columns.tolist() :

        
#         index_nan.append(dataframe.loc[dataframe[column] > dataframe[
#             column].quantile(0.99)].index.tolist())
#         index_nan.append(dataframe.loc[dataframe[column] < dataframe[
#             column].quantile(0.01)].index.tolist())

#     for sublist in index_nan:
#         for item in sublist:
#             index_nan_flat.append(item)
                
#     #suppression des doublons
#     index_nan_flat = list(dict.fromkeys(index_nan_flat))
#     dataframe[column].loc[index_nan_flat] = np.nan

#     return dataframe.dropna(axis=0)

# def delete_multivariate_outliers(dataframe):
#     '''Suppression des outliers multivariés 
#     (1% le plus éloigné par le calcul de la distance aux 5 plus proches voisins)
#     Entree : objet dataframe
#     Sortie : objet dataframe
#     '''
    
#     lof = LocalOutlierFactor(n_neighbors = 5, n_jobs=-1)
#     lof.fit_predict(dataframe.select_dtypes(['float64','int32']).dropna())
#     indices = dataframe.select_dtypes(['float64','int32']).dropna().index
#     df_lof = pd.DataFrame(index = indices,
#                            data = lof.negative_outlier_factor_, columns=['lof'])
#     index_to_drop = df_lof[df_lof['lof']< np.quantile(
#         lof.negative_outlier_factor_, 0.01)].index
#     return dataframe.drop(index_to_drop, axis=0)

# def clean_outliers(dataframe):
#     dataframe = delete_univariate_outliers(dataframe)

#     dataframe = delete_multivariate_outliers(dataframe)
#     return dataframe


In [ ]:
# Importation de la fonction LocalOutlierFactor du module sklearn.neighbors
from sklearn.neighbors import LocalOutlierFactor

# Définition d'une fonction pour supprimer les valeurs extrêmes univariées d'un dataframe
def delete_univariate_outliers(dataframe):
    '''Suppression des valeurs extrêmes du dataset - on exclut le centile le plus extreme
    Entree: objet dataframe
    Traitement : Supression Nan univariés
    Sortie : objet dataframe
    '''
    # Initialisation d'une liste pour stocker les index des valeurs extrêmes
    index_nan = []
    index_nan_flat = []
    
    # Parcours des colonnes numériques du dataframe
    for column in dataframe.select_dtypes(include=['int32', 'float64']).columns.tolist():

        # Recherche des index des valeurs supérieures au centile 99 et inférieures au centile 1
        index_nan.append(dataframe.loc[dataframe[column] > dataframe[column].quantile(0.99)].index.tolist())
        index_nan.append(dataframe.loc[dataframe[column] < dataframe[column].quantile(0.01)].index.tolist())

    # Aplatir la liste des index des valeurs extrêmes
    for sublist in index_nan:
        for item in sublist:
            index_nan_flat.append(item)
                
    # Suppression des doublons dans la liste des index
    index_nan_flat = list(dict.fromkeys(index_nan_flat))
    
    # Remplacement des valeurs extrêmes par NaN dans le dataframe
    for column in dataframe.select_dtypes(include=['int32', 'float64']).columns.tolist():
        dataframe[column].loc[index_nan_flat] = np.nan

    # Suppression des lignes contenant des valeurs NaN
    return dataframe.dropna(axis=0)

# Définition d'une fonction pour supprimer les outliers multivariés d'un dataframe
def delete_multivariate_outliers(dataframe):
    '''Suppression des outliers multivariés 
    (1% le plus éloigné par le calcul de la distance aux 5 plus proches voisins)
    Entree : objet dataframe
    Sortie : objet dataframe
    '''
    # Création d'un modèle Local Outlier Factor (LOF) avec 5 voisins
    lof = LocalOutlierFactor(n_neighbors=5, n_jobs=-1)
    
    # Entraînement du modèle et prédiction des outliers
    lof.fit_predict(dataframe.select_dtypes(['float64', 'int32']).dropna())
    
    # Récupération des indices des outliers
    indices = dataframe.select_dtypes(['float64', 'int32']).dropna().index
    df_lof = pd.DataFrame(index=indices, data=lof.negative_outlier_factor_, columns=['lof'])
    
    # Suppression des outliers dont le score LOF est inférieur au 1er centile
    index_to_drop = df_lof[df_lof['lof'] < np.quantile(lof.negative_outlier_factor_, 0.01)].index
    return dataframe.drop(index_to_drop, axis=0)

# Définition d'une fonction pour nettoyer les outliers d'un dataframe
def clean_outliers(dataframe):
    # Suppression des outliers univariés
    dataframe = delete_univariate_outliers(dataframe)

    # Suppression des outliers multivariés
    dataframe = delete_multivariate_outliers(dataframe)
    return dataframe


Ces fonctions permettent de nettoyer les valeurs aberrantes dans un dataframe en supprimant les valeurs extrêmes univariées et les outliers multivariés. Les valeurs extrêmes univariées sont supprimées en remplaçant les valeurs qui dépassent les centiles 1 et 99 par des NaN. Les outliers multivariés sont identifiés en utilisant le modèle LOF (Local Outlier Factor) et sont supprimés s'ils ont un score LOF inférieur au 1er centile. Le dataframe nettoyé est ensuite renvoyé.

fonction de nettoyage globale

In [ ]:
# Définition d'une fonction pour appliquer le traitement complet et créer de nouvelles features à partir d'une liste de données
def apply_features(liste_donnees):
    '''Application traitement complet et création de nouvelles features
    
    Traitement:
    * Création de nouvelles features
    * Merge de l'ensemble des dataframe dans un seul dataframe sur la clé client unique
    * Nettoyage des outliers
    
    Entree : liste d'objets 
    Sortie : dataframe cleané
    '''
    print('### Création features ###')
    # Récupération des dataframes à partir de la liste de données
    customers_local = liste_donnees[0].copy()
    geolocalisation_local = liste_donnees[1].copy()
    order_items_local = liste_donnees[2].copy()
    order_payments_local = liste_donnees[3].copy()
    order_reviews_local = liste_donnees[4].copy()
    orders_local = liste_donnees[5].copy()
    products_local = liste_donnees[6].copy()
    sellers_local = liste_donnees[7].copy()
    translation_local = liste_donnees[8].copy()
    
    # Nombre de produits achetés par client
    produits_par_client = pd.merge(customers_local, pd.merge(order_items_local, orders_local))
    nb_produits = produits_par_client.groupby('customer_id')['product_id'].count()
    nb_produits.rename('Nb_pdts', inplace=True)
    
    # 100 premières villes
    first_cities = customers_local.groupby(['customer_city']).count()['customer_state'].sort_values(ascending=False).head(100).index.tolist()
    index_cities = customers_local[~customers_local['customer_city'].isin(first_cities)].index
    customers_local.loc[index_cities, 'customer_city'] = 'Other'
    
    # Catégorie la plus achetée
    cat = pd.merge(produits_par_client, products_local).sort_values(['customer_id','product_category_name_english'], ascending=False).groupby(['customer_id','product_category_name_english']).head(1)[['customer_unique_id', 'product_category_name_english']]
    cat.columns = ['customer_unique_id', 'Cat_la_plus_achetee']
    cat.set_index('customer_unique_id', inplace=True)

    # Montant moyen des achats
    achats_moy = pd.merge(order_items_local, orders_local).groupby(['customer_id', 'order_id'])['price'].sum().groupby(['customer_id']).mean()
    achats_moy.rename('Tot_moy_achats', inplace=True)

    # Montant maximum des achats
    achats_max = pd.merge(order_items_local, orders_local).groupby(['customer_id', 'order_id'])['price'].max().groupby(['customer_id']).max()
    achats_max.rename('Mont_max_achats', inplace=True)
    
    # Nombre moyen de produits par commande
    nb_moyen_prod = pd.merge(order_items_local, orders_local).groupby(['customer_id', 'order_id'])['price'].count().groupby(['customer_id']).mean()
    nb_moyen_prod.rename('Nb_moy_pdts_par_com', inplace=True)
    
    # Délai moyen de livraison
    delai_delivery = pd.merge(orders_local, order_items_local)
    delai_delivery.set_index('customer_id', inplace=True)
    delai_delivery = delai_delivery['order_delivered_customer_date'] - delai_delivery['order_purchase_timestamp']
    delai_delivery.rename('Delai_Moy_Commande', inplace=True)
    delai_delivery = delai_delivery.dt.days + 1
    
    # Heure du dernier achat
    date_achat = pd.merge(order_items_local, orders_local)[['customer_id', 'order_purchase_timestamp']].groupby(['customer_id']).max()
    date_achat['heure_achat'] = date_achat['order_purchase_timestamp'].apply(lambda x: x.hour)

    # Jour de la semaine du dernier achat
    date_achat['jour_achat'] = date_achat['order_purchase_timestamp'].apply(lambda x: x.weekday())

    # Nombre de jours écoulés depuis le dernier achat
    date_achat['delai_dernier_achat'] = date_achat['order_purchase_timestamp'].max() -  date_achat['order_purchase_timestamp']
    date_achat['delai_dernier_achat'] = date_achat['delai_dernier_achat'].apply(lambda x: x.days)
    
    # Note moyenne des commentaires
    note_moy = pd.merge(order_reviews_local, orders_local).groupby('customer_id')['review_score'].mean()
    note_moy.rename('Note_Moy_Com', inplace=True)
    
    # Moyen de paiement le plus utilisé
    paiement = pd.merge(orders_local, order_payments_local).sort_values(['customer_id','payment_type','payment_installments'], ascending=False).groupby(['customer_id', 'payment_type','payment_installments']).head(1)[['customer_id', 'payment_type','payment_installments']]
    paiement.columns = ['customer_id', 'Moy_Paiment','Facilités']
    paiement.set_index('customer_id', inplace=True)

    customers_local.set_index('customer_id', inplace=True)
    
    # Merge des nouvelles features avec le dataframe customers_local
    for df in [nb_produits, achats_moy, achats_max, delai_delivery, nb_moyen_prod, date_achat, note_moy, paiement]:
        customers_local = pd.merge(left=customers_local, right=df, how='left', left_index=True, right_index=True)

    customers_local.reset_index(inplace=True)
    cat.reset_index(inplace=True)
    
    # Conversion des colonnes customer_id en type 'object'
    customers_local['customer_id'] = customers_local['customer_id'].astype('object')
    cat['customer_unique_id'] = cat['customer_unique_id'].astype('object')
    
    # Merge du dataframe customers_local avec la catégorie la plus achetée
    customers_local = pd.merge(left=cat, right=customers_local, on='customer_unique_id', how='right')
    customers_local.reset_index(inplace=True)
    
    print('### Cleaning NaN ###')
    # Cleaning des NaN des features
    customers_local['Cat_la_plus_achetee'].fillna('Unknown', inplace=True)
    customers_local[['Nb_pdts', 'Tot_moy_achats', 'Mont_max_achats', 'Nb_moy_pdts_par_com']].dropna(how='all', axis=0, inplace=True)
    customers_local['Delai_Moy_Commande'].fillna(-1, inplace=True)
    customers_local.dropna(axis=0, inplace=True)

    print('### Aggrégation des features sur id client unique ###')
    # Agrégation des clients par identifiant unique
    dict_agg = {
        'Cat_la_plus_achetee': lambda x: x.mode()[0],
        'customer_zip_code_prefix': lambda x: x.mode()[0],
        'customer_city': lambda x: x.mode()[0],
        'customer_state': lambda x: x.mode()[0],
        'Nb_pdts': 'sum',
        'Tot_moy_achats': 'mean',
        'Mont_max_achats': 'max',
        'Delai_Moy_Commande': 'mean',
        'Nb_moy_pdts_par_com': 'mean',
        'order_purchase_timestamp': 'max',
        'heure_achat': lambda x: x.mode()[0],
        'jour_achat': lambda x: x.mode()[0],
        'delai_dernier_achat': 'min',
        'Note_Moy_Com': 'mean',
        'Moy_Paiment': lambda x: x.mode()[0],
        'Facilités': 'mean',
        'geolocation_lat': 'mean',
        'geolocation_lng': 'mean',
        ('price', 'home'): 'sum',
        ('price', 'appliances'): 'sum',
        ('price', 'construction'): 'sum',
        ('price', 'office'): 'sum',
        ('price', 'electronics'): 'sum',
        ('price', 'arts'): 'sum',
        ('price', 'fashion'): 'sum',
        ('price', 'health_beauty'): 'sum',
        ('price', 'sports_leisure'): 'sum',
        ('price', 'garden_pets'): 'sum',
        ('price', 'auto'): 'sum',
        ('price', 'food_drinks'): 'sum',
        ('price', 'other'): 'sum'
    }

    print('### Ajout des dépenses par catégories ###')
    # Montant dépensé par catégorie de produits
    table_cat = pd.merge(customers_local, pd.merge(pd.merge(products_local, translation_local), pd.merge(order_items_local, orders_local)))[['customer_unique_id', 'product_category_name_english', 'price']]
    table_cat['product_category_name_english'] = table_cat['product_category_name_english'].map(dict_categories)
    table_cat = pd.pivot_table(table_cat, index='customer_unique_id', columns='product_category_name_english', aggfunc=np.sum, fill_value=0).reset_index()
    customers_local = pd.merge(customers_local, table_cat, left_on='customer_unique_id', right_on='customer_unique_id')
    
    # Merge des informations géographiques
    geol = pd.merge(customers_local, geolocalisation_local, how='left', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix').groupby('customer_unique_id').mean()[['geolocation_lat', 'geolocation_lng']].reset_index()
    customers_local = pd.merge(customers_local, geol, left_on='customer_unique_id', right_on='customer_unique_id')
    
    # Agrégation finale par identifiant unique du client
    customers_local = customers_local.groupby('customer_unique_id').agg(dict_agg)
    customers_local['Cat_la_plus_achetee'] = customers_local['Cat_la_plus_achetee'].map(dict_categories)
    
    print('### Outliers ###')
    # Nettoyage des outliers
    print('taille du jeu de données pré-nettoyage outliers : ', customers_local.shape)
    customers_local = clean_outliers(customers_local)
    print('taille du jeu de données post-nettoyage outliers : ', customers_local.shape)
    
    print('### Log transformation ###')
    # Transformation log
    columns_log = ['Tot_moy_achats', 'Mont_max_achats']
    customers_local[columns_log] = np.log(customers_local[columns_log])
    return customers_local

Explication étape par étape du code `apply_features` :

1. **Création des nouvelles features :**
   - Le code commence par créer plusieurs nouvelles features à partir des différentes tables de données (`customers_local`, `order_items_local`, `orders_local`, `order_reviews_local`, etc.).
   - Par exemple, il calcule le nombre total de produits achetés par client (`Nb_pdts`), le montant moyen des achats (`Tot_moy_achats`), le montant maximum dépensé par client (`Mont_max_achats`), le délai moyen de livraison des commandes (`Delai_Moy_Commande`), le nombre moyen de produits par commande (`Nb_moy_pdts_par_com`), etc.
   - Il récupère également l'heure, le jour de la semaine et le nombre de jours écoulés depuis le dernier achat (`heure_achat`, `jour_achat`, `delai_dernier_achat`).
   - Il calcule la note moyenne des commentaires laissés par les clients (`Note_Moy_Com`) et identifie le moyen de paiement le plus utilisé par chaque client (`Moy_Paiment`).
   - Il ajoute aussi des coordonnées géographiques (`geolocation_lat`, `geolocation_lng`) moyennes basées sur le code postal du client.

2. **Nettoyage des NaN et Outliers :**
   - Ensuite, le code effectue un nettoyage en supprimant les NaN dans les features nouvellement créées.
   - Il supprime également les outliers (valeurs extrêmes) en utilisant deux fonctions `clean_outliers` : `delete_univariate_outliers` pour les valeurs extrêmes univariées et `delete_multivariate_outliers` pour les valeurs extrêmes multivariées.

3. **Transformation log :**
   - Certaines colonnes du dataframe sont ensuite transformées en prenant leur logarithme (`np.log`) pour mieux distribuer les valeurs et réduire l'effet des valeurs extrêmes.

4. **Agrégation des données :**
   - Une fois les features créées, nettoyées et transformées, le code procède à une agrégation des clients par identifiant unique (`customer_unique_id`).
   - Il groupe les clients par cette clé unique et applique des fonctions d'agrégation pour consolider les valeurs des différentes features. Par exemple, pour certaines features, il prend la valeur moyenne (`mean`), pour d'autres, il somme les valeurs (`sum`), et pour d'autres encore, il prend le maximum (`max`).

5. **Ajout des dépenses par catégories :**
   - Le code calcule ensuite les montants dépensés par chaque client dans chaque catégorie de produits (`home`, `appliances`, `construction`, etc.).
   - Il utilise une table pivot (`pd.pivot_table`) pour regrouper les montants dépensés par catégorie.

6. **Merge des informations géographiques :**
   - Les informations géographiques (latitude et longitude) sont ensuite ajoutées au dataframe principal en utilisant le code postal du client pour effectuer un merge avec la table `geolocalisation_local`.

7. **Agrégation finale et nettoyage :**
   - Enfin, le code effectue une dernière agrégation en utilisant les mêmes fonctions d'agrégation que précédemment pour consolider les informations géographiques.
   - Il convertit également la catégorie la plus achetée (`Cat_la_plus_achetee`) en utilisant un dictionnaire de correspondance (`dict_categories`) pour des noms plus lisibles.
   - Enfin, il effectue un dernier nettoyage en supprimant les éventuels NaN restants et les outliers, et retourne le dataframe nettoyé et agrégé.

Le résultat final est un dataframe contenant toutes les informations agrégées par client, prêtes à être utilisées pour des analyses ultérieures ou pour construire un modèle de machine learning.

###Création features###
###Cleaning NaN###
###Aggrégation des features sur id client unique###
###Ajout des dépenses par catégories###
###Outliers###
taille du jeu de données pré-nettoyage outliers :  (93396, 31)
taille du jeu de données post-nettoyage outliers :  (74705, 31)
###Log transformation###

In [ ]:
# data_clients = apply_features(liste_df)

In [ ]:
# data_clients.head()

In [ ]:
# data_clients.columns

# Exploration 

## informations jeu de données initial

**Historique**

In [ ]:
orders['order_purchase_timestamp'].max()

In [ ]:
orders['order_purchase_timestamp'].min()

On a 2 années d'historique : de septembre 2016 à octobre 2018

**Nombre de clients**

In [ ]:
customers['customer_unique_id'].nunique()

**Nombre de transactions**

In [ ]:
orders['order_id'].nunique()

On a presque autant de clients que de transactions => peu de clients ont fait plus d'une transaction

**Nombre de vendeurs**

In [ ]:
sellers['seller_id'].nunique()

In [ ]:
sellers # ZIPCODE

**Nombre de clients par vendeur**

In [ ]:
pd.merge(order_items, orders).groupby(
    ['seller_id','customer_id']).count().reset_index().groupby(
    'seller_id').count()['customer_id'].sum()

## jeu de donnée préparé

### Distribution des variables

In [ ]:
%matplotlib inline
for column in data_clients.select_dtypes(['int32', 'float64']).columns:
    f, axes = plt.subplots(1,2, figsize=(12,4))
    titre = 'Distribution de ' + str(column)
    plt.title(titre)
    sns.distplot(data_clients[column], bins=30, ax=axes[0])
    titre = 'Distribution de ' + str(column)
    plt.title(titre)
    sns.boxplot(data_clients[column], ax=axes[1])
    plt.show()

In [ ]:
%matplotlib inline
for column in data_clients.select_dtypes(['int32', 'float64']).columns:
    if 'price' in column:
        f, axes = plt.subplots(1,2, figsize=(12,4))
        titre = 'Distribution de ' + str(column) +' sans valeur nulle'
        plt.title(titre)
        sns.distplot(data_clients[data_clients[column] != 0][column], bins=30, ax=axes[0])
        titre = 'Distribution de ' + str(column) +'sans valeur nulle'
        plt.title(titre)
        sns.boxplot(data_clients[data_clients[column] != 0][column], ax=axes[1])
        plt.show()

In [ ]:
state = data_clients.groupby(['customer_state']).count().sort_values(
    by='customer_city', ascending=False)['customer_city'].head(10)
plt.figure(figsize=(10,6))
plt.title('Les 10 états avec le plus de clients')
sns.barplot(x = state.values,
           y = state.index)

In [ ]:
data_clients

In [ ]:
data_clients['Cat_la_plus_achetee'].unique()

In [ ]:
products['product_weight_g'].mean()

In [ ]:
cities = data_clients.groupby(['customer_city']).count()[
    'customer_state'].sort_values(ascending=False).head(10)
plt.figure(figsize=(10,6))
plt.title('Les 10 villes avec le plus de clients')
sns.barplot(x = cities.values,
           y = cities.index)

In [ ]:
categories = data_clients.groupby(['Cat_la_plus_achetee']).count()[
    'customer_city'].sort_values(ascending=False)
plt.figure(figsize=(10,8))
plt.title('Répartition des catégories les plus achetées par les clients')
sns.barplot(x = categories.values,
           y = categories.index)

In [ ]:
payment = data_clients.groupby(['Moy_Paiment']).count()[
    'customer_city'].sort_values(ascending=False)
plt.figure(figsize=(10,4))
plt.title('Répartition des moyens de paiement plébiscités par les clients')
sns.barplot(x = payment.values,
           y = payment.index)

In [ ]:
delai_achat = data_clients['order_purchase_timestamp'].max() - data_clients[
    'order_purchase_timestamp']
plt.figure(figsize=(8,6))
plt.title('Nombre de jours écoulés depuis la dernière commande')
sns.distplot(delai_achat.dt.days, bins=50)
plt.show()

**Conclusions sur l'analyse des distributions des variables:**
* très peu de clients ont fait plus d'un achat;
* les distributions des montants d'achat par catégorie sont d'allure exponentielle
* la variable price food drinks n'apporte pas d'information

### Corrélations

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data_clients.corr())

In [ ]:
# Utilisation de la fonction avec votre DataFrame 'data_clients'
show_correlogram(data_clients)

In [ ]:
# Sélectionner les colonnes numériques
numeric_columns = data_clients.select_dtypes(['int32', 'float64', 'int8', 'float32']).columns
print(numeric_columns)
pears_corr = data_clients[numeric_columns].corr()
plot_heatmap(pears_corr, shape='tri', title="Pearson correlation", figsize=(12, 5))
plt.show()

On voit que certaines variables sont corrélées compte tenu du faible nombre de clients qui ont plusieurs transactions. C'est le cas de :
* nb de produits achetés avec nombre moyen de produits par commande
* montant total des achats avec montant moyen par commande

Egalement, la feature ('price', 'food_drinks') ne contient pas d'information

On va conserver une feature sur les 2 pour chaque corrélation

In [ ]:
data_clients.drop(['Mont_max_achats','Nb_moy_pdts_par_com'], axis=1, inplace=True)

In [ ]:
data_clients

# Export jeu de données 

In [ ]:
data_clients.select_dtypes(['object']).nunique()

In [ ]:
data_clients.to_csv('data/data_cleaned.csv')

In [ ]:
data_clients.sample(10)

In [ ]:
data_clients.describe()

In [ ]:
# import folium

# # Création d'une carte centrée sur la première ligne de vos données (par exemple)
# latitude_centre = geolocalisation['geolocation_lat'].iloc[0]
# longitude_centre = geolocalisation['geolocation_lng'].iloc[0]
# carte = folium.Map(location=[latitude_centre, longitude_centre], zoom_start=10)

# # Ajout de marqueurs pour chaque emplacement (latitude, longitude) dans vos données
# for index, row in geolocalisation.iterrows():
#     latitude = row['geolocation_lat']
#     longitude = row['geolocation_lng']
#     city = row['geolocation_city']
#     popup_text = f"Emplacement {index}: {city}"  # Texte affiché dans le popup (avec la ville)
#     folium.Marker([latitude, longitude], popup=popup_text).add_to(carte)

# # Affichage de la carte
# carte


In [ ]:
geolocalisation

In [ ]:
# # Import de la librairie folium pour créer une carte interactive
# import folium

# # Création d'une carte centrée sur la position moyenne des points
# m = folium.Map(location=[geolocalisation['geolocation_lat'].mean(), geolocalisation['geolocation_lng'].mean()], zoom_start=11)

# # Boucle sur toutes les lignes du dataset
# for i in range(0,len(geolocalisation)):
#     # Ajout d'un cercle sur la carte à la position (latitude, longitude) de chaque ligne
#     # avec un rayon de 100 mètres et un popup affichant l'adresse
#     folium.Circle([geolocalisation.iloc[i]['geolocation_lat'],geolocalisation.iloc[i]['geolocation_lng']], popup=geolocalisation.iloc[i]['geolocation_city'], radius=100).add_to(m)

# # Affichage de la carte interactive
# m


In [ ]:
# from ipyleaflet import Map, Marker

# # Création d'une carte centrée sur la position moyenne des points
# m = Map(center=[geolocalisation['geolocation_lat'].mean(), geolocalisation['geolocation_lng'].mean()], zoom=11)

# # Boucle sur toutes les lignes du GeoDataFrame
# for index, row in geolocalisation.iterrows():
#     # Ajout d'un marqueur à chaque position géographique
#     marker = Marker(location=[row['geolocation_lat'], row['geolocation_lng']], title=row['geolocation_city'])
#     m.add_layer(marker)

# # Affichage de la carte interactive
# mf


In [ ]:
# import folium

# # Création d'une nouvelle colonne 'coordinates' contenant les coordonnées géographiques sous forme de liste
# geolocalisation['coordinates'] = geolocalisation.apply(lambda row: [row['geolocation_lat'], row['geolocation_lng']], axis=1)

# # Création de la carte centrée sur la position moyenne des points
# m = folium.Map(location=[geolocalisation['geolocation_lat'].mean(), geolocalisation['geolocation_lng'].mean()], zoom_start=11)

# # Boucle sur toutes les lignes du dataset
# for index, row in geolocalisation.iterrows():
#     # Ajout d'un cercle sur la carte à la position (latitude, longitude) de chaque ligne
#     # avec un rayon de 100 mètres et un popup affichant l'adresse
#     folium.Circle(location=row['coordinates'], popup=row['geolocation_city'], radius=100).add_to(m)

# # Affichage de la carte interactive
# m


In [ ]:
data_clients

In [ ]:
data_clients.columns

In [ ]:
import pandas as pd

# Afficher l'index actuel du DataFrame data_clients
print(data_clients.index)


In [ ]:
import pandas as pd

# Réinitialiser l'index pour que 'customer_unique_id' redevienne une colonne du DataFrame
data_clients_reset = data_clients.reset_index()

# Calcul de la récence
recency_df = data_clients_reset[['customer_unique_id', 'order_purchase_timestamp']]
recency_df['order_purchase_timestamp'] = pd.to_datetime(recency_df['order_purchase_timestamp'])
max_date = recency_df['order_purchase_timestamp'].max()
recency_df['recency'] = (max_date - recency_df['order_purchase_timestamp']).dt.days

# Supprimer la colonne 'order_purchase_timestamp' car elle n'est plus nécessaire
recency_df.drop(columns=['order_purchase_timestamp'], inplace=True)

# Calcul de la fréquence et de la valeur monétaire
frequency_df = data_clients_reset.groupby('customer_unique_id').agg({'order_purchase_timestamp': 'count', 'Tot_moy_achats': 'sum'})
# frequency_df.rename(columns={'order_purchase_timestamp': 'frequency', 'Mont_max_achats': 'monetary_value'}, inplace=True)
frequency_df.rename(columns={'order_purchase_timestamp': 'frequency', 'Tot_moy_achats': 'monetary_value'}, inplace=True)

# Concaténer les trois mesures : récence, fréquence et valeur monétaire
rfm_df = pd.concat([recency_df.set_index('customer_unique_id'), frequency_df], axis=1)

# Affichage des 5 premières lignes du DataFrame rfm_df
print(rfm_df.head())


In [ ]:
import matplotlib.pyplot as plt

# Création d'un diagramme en nuage de points (scatter plot)
plt.figure(figsize=(10, 6))
plt.scatter(rfm_df['recency'], rfm_df['frequency'], c=rfm_df['monetary_value'], cmap='viridis', alpha=0.8)
plt.colorbar(label='Valeur monétaire')
plt.xlabel('Récence (jours)')
plt.ylabel('Fréquence')
plt.title('Segmentation RFM')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Création d'un graphique en 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Ajout des points sur le graphique en 3D
ax.scatter(rfm_df['recency'], rfm_df['frequency'], rfm_df['monetary_value'], c=rfm_df['monetary_value'], cmap='viridis', alpha=0.8)

# Configuration des axes
ax.set_xlabel('Récence (jours)')
ax.set_ylabel('Fréquence')
ax.set_zlabel('Valeur monétaire')
ax.set_title('Segmentation RFM')

# Ajout de la barre de couleur pour représenter la valeur monétaire
cbar = plt.colorbar(ax.scatter([], [], [], c=[], cmap='viridis'), ax=ax)
cbar.set_label('Valeur monétaire')

plt.show()


In [ ]:
# # Utilisation de Folium :

# import folium

# # Création d'une nouvelle colonne 'coordinates' contenant les coordonnées géographiques sous forme de liste
# geolocalisation['coordinates'] = geolocalisation.apply(lambda row: [row['geolocation_lat'], row['geolocation_lng']], axis=1)

# # Création de la carte centrée sur la position moyenne des points
# m = folium.Map(location=[geolocalisation['geolocation_lat'].mean(), geolocalisation['geolocation_lng'].mean()], zoom_start=11)

# # Boucle sur toutes les lignes du dataset
# for index, row in geolocalisation.iterrows():
#     # Ajout d'un cercle sur la carte à la position (latitude, longitude) de chaque ligne
#     # avec un rayon de 100 mètres et un popup affichant l'adresse
#     folium.Circle(location=row['coordinates'], popup=row['geolocation_city'], radius=100).add_to(m)

# # Affichage de la carte interactive
# m

In [ ]:
# # Utilisation de Geopandas (pour les données géospatiales) :

# import geopandas as gpd
# import matplotlib.pyplot as plt

# # Création du GeoDataFrame à partir du DataFrame geolocalisation
# gdf = gpd.GeoDataFrame(
#     geolocalisation, 
#     geometry=gpd.points_from_xy(geolocalisation['geolocation_lng'], geolocalisation['geolocation_lat'])
# )

# # Création de la carte centrée sur la position moyenne des points
# ax = gdf.plot(figsize=(10, 10), alpha=0.5, markersize=10)

# # Affichage de la carte
# plt.show()

In [ ]:
# # Utilisation de Geopandas (pour les données géospatiales) :

# import geopandas as gpd
# import matplotlib.pyplot as plt

# # Création du GeoDataFrame à partir du DataFrame geolocalisation
# gdf = gpd.GeoDataFrame(
#     data_clients, 
#     geometry=gpd.points_from_xy(data_clients['geolocation_lng'], data_clients['geolocation_lat'])
# )

# # Création de la carte centrée sur la position moyenne des points
# ax = gdf.plot(figsize=(10, 10), alpha=0.5, markersize=10)

# # Affichage de la carte
# plt.show()

In [ ]:
# import folium

# # Création de la carte centrée sur la position moyenne des points
# m = folium.Map(location=[geolocalisation['geolocation_lat'].mean(), geolocalisation['geolocation_lng'].mean()], zoom_start=11)

# # Boucle sur toutes les lignes du GeoDataFrame
# for index, row in gdf.iterrows():
#     # Ajout d'un marqueur à chaque position géographique
#     folium.Marker([row['geolocation_lat'], row['geolocation_lng']], popup=row['geolocation_city']).add_to(m)

# # Affichage de la carte interactive
# m


In [ ]:
# # Utilisation de Plotly (Mapbox) :
# import plotly.graph_objects as go

# # Création de la carte centrée sur la position moyenne des points
# m = go.Figure(go.Scattermapbox(
#     lat=geolocalisation['geolocation_lat'],
#     lon=geolocalisation['geolocation_lng'],
#     mode='markers',
#     marker=go.scattermapbox.Marker(size=10),
#     text=geolocalisation['geolocation_city'],
# ))

# m.update_layout(
#     mapbox_style='open-street-map',
#     mapbox_center={'lat': geolocalisation['geolocation_lat'].mean(), 'lon': geolocalisation['geolocation_lng'].mean()},
#     mapbox_zoom=11,
# )

# # Affichage de la carte interactive
# m.show()

In [ ]:
# Importer la bibliothèque Plotly pour créer une carte interactive
import plotly.graph_objects as go

# Création de la carte centrée sur la position moyenne des points
m = go.Figure(go.Scattermapbox(
    lat=data_clients['geolocation_lat'],      # Coordonnées de latitude des points
    lon=data_clients['geolocation_lng'],      # Coordonnées de longitude des points
    mode='markers',                           # Mode d'affichage des marqueurs (ici, des points)
    marker=go.scattermapbox.Marker(size=10),  # Taille des marqueurs (points) sur la carte
    text=data_clients['customer_city'],       # Texte affiché lorsqu'on survole un point (ville du client)
))

# Mise à jour du style de la carte avec une carte de fond "open-street-map"
# Définir une variable pour stocker le niveau de zoom souhaité
zoom_level = 2

m.update_layout(
    mapbox_style='open-street-map',
    
    # Centrage de la carte sur la position moyenne des points (latitude et longitude)
    mapbox_center={'lat': data_clients['geolocation_lat'].mean(), 'lon': data_clients['geolocation_lng'].mean()},
    
    # Utiliser la variable 'zoom_level' pour définir le niveau de zoom initial de la carte
    mapbox_zoom=zoom_level,
)

# Affichage de la carte interactive
m.show()


In [ ]:
# import folium

# # Création de la carte centrée sur la position moyenne des points
# m = folium.Map(location=[data_clients['geolocation_lat'].mean(), data_clients['geolocation_lng'].mean()], zoom_start=11)

# # Boucle sur toutes les lignes du GeoDataFrame
# for index, row in gdf.iterrows():
#     # Ajout d'un marqueur à chaque position géographique
#     folium.Marker([row['geolocation_lat'], row['geolocation_lng']], popup=row['customer_city']).add_to(m)

# # Affichage de la carte interactive
# m


In [ ]:
# fig, ax = plt.subplots(1)
# sns.scatterplot(data_clients['geolocation_lat'], data_clients['geolocation_lng'],
#                 s=5, hue = np.log(data_clients['customer_city']),
#                 alpha=0.7, marker='o', ec=None, palette='viridis')
# ax.set_axis_off()
# ax.set_aspect('equal', adjustable='box')
# ax.legend(loc=3)
# fig.set_size_inches(10,10)
# fig.savefig('customers_loc.png', transparent=True)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Charger les données depuis les fichiers CSV
customers = pd.read_csv('data/olist_customers_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')

# Fusionner les DataFrames pour avoir toutes les informations nécessaires en un seul DataFrame
df = orders.merge(customers, on='customer_id')
df = df.merge(order_items, on='order_id')
df = df.merge(order_payments, on='order_id')

# Convertir la colonne 'order_purchase_timestamp' en type datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Calculer la date de la dernière commande (récence)
current_date = df['order_purchase_timestamp'].max()
df['recency'] = (current_date - df['order_purchase_timestamp']).dt.days

# Calculer la fréquence des commandes par client
frequency_df = df.groupby('customer_unique_id')['order_id'].nunique().reset_index()
frequency_df.rename(columns={'order_id': 'frequency'}, inplace=True)
df = df.merge(frequency_df, on='customer_unique_id', how='left')

# Calculer le montant total dépensé par client
df['total_spent'] = df['payment_value']

# Maintenant que nous avons les colonnes Récence, Fréquence et Montant, nous pouvons appliquer la méthode RFM

# Fonction pour attribuer un score à chaque dimension RFM en fonction des quartiles
def assign_rfm_score(x, quartiles):
    if x <= quartiles[0]:  # Quartile le plus bas
        return 1
    elif x <= quartiles[1]:  # Deuxième quartile
        return 2
    elif x <= quartiles[2]:  # Troisième quartile
        return 3
    else:  # Quartile le plus élevé
        return 4

# Calculer les quartiles pour chaque dimension RFM
recency_quartiles = df['recency'].quantile([0.25, 0.5, 0.75]).values
frequency_quartiles = df['frequency'].quantile([0.25, 0.5, 0.75]).values
monetary_quartiles = df['total_spent'].quantile([0.25, 0.5, 0.75]).values

# Assigner les scores RFM à chaque client
df['R'] = df['recency'].apply(assign_rfm_score, args=(recency_quartiles,))
df['F'] = df['frequency'].apply(assign_rfm_score, args=(frequency_quartiles,))
df['M'] = df['total_spent'].apply(assign_rfm_score, args=(monetary_quartiles,))

# Calculer le score RFM total en concaténant les scores R, F et M
df['RFM_Score'] = df['R'].astype(str) + df['F'].astype(str) + df['M'].astype(str)

# Maintenant, vous avez le score RFM pour chaque client dans la colonne 'RFM_Score'
# Vous pouvez utiliser ce score pour segmenter vos clients en différentes catégories RFM.

# Par exemple, les clients ayant un score RFM de '111' sont considérés comme les plus précieux,
# tandis que ceux ayant un score de '444' sont considérés comme les moins précieux.

# Vous pouvez également attribuer des étiquettes (comme 'Platinum', 'Gold', 'Silver', 'Bronze', etc.)
# en fonction des scores RFM pour identifier les groupes de clients avec différentes caractéristiques de comportement d'achat.

# Pour analyser davantage les segments RFM, vous pouvez utiliser des graphiques, des tableaux croisés dynamiques, etc.

# Exemple de segmentation RFM simple :
def segment_rfm(df):
    if df['RFM_Score'] == '111':
        return 'Premium'
    elif df['RFM_Score'] == '444':
        return 'Low Value'
    else:
        return 'Standard'

# Appliquer la segmentation RFM au DataFrame
df['RFM_Segment'] = df.apply(segment_rfm, axis=1)

# Vous pouvez également enregistrer les résultats dans un nouveau fichier CSV, si nécessaire
df.to_csv('resultats_rfm.csv', index=False)

# Créer un graphique pour visualiser la répartition des segments RFM
plt.figure(figsize=(10, 6))
df['RFM_Segment'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Répartition des segments RFM')
plt.xlabel('Segments RFM')
plt.ylabel('Nombre de clients')
plt.xticks(rotation=0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Création d'un diagramme en nuage de points (scatter plot)
plt.figure(figsize=(10, 6))
plt.scatter(df['recency'], df['frequency'], c=df['total_spent'], cmap='viridis', alpha=0.8)
plt.colorbar(label='Valeur monétaire')
plt.xlabel('Récence (jours)')
plt.ylabel('Fréquence')
plt.title('Segmentation RFM')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# # Assigner les scores RFM à chaque client
# df['R'] = df['recency'].apply(assign_rfm_score, args=(recency_quartiles,))
# df['F'] = df['frequency'].apply(assign_rfm_score, args=(frequency_quartiles,))
# df['M'] = df['total_spent'].apply(assign_rfm_score, args=(monetary_quartiles,))
# Création d'un graphique en 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Ajout des points sur le graphique en 3D
ax.scatter(df['recency'], df['frequency'], df['total_spent'], c=df['total_spent'], cmap='viridis', alpha=0.8)

# Configuration des axes
ax.set_xlabel('Récence (jours)')
ax.set_ylabel('Fréquence')
ax.set_zlabel('Valeur monétaire')
ax.set_title('Segmentation RFM')

# Ajout de la barre de couleur pour représenter la valeur monétaire
cbar = plt.colorbar(ax.scatter([], [], [], c=[], cmap='viridis'), ax=ax)
cbar.set_label('Valeur monétaire')

plt.show()


In [ ]:
# import pandas as pd
# import plotly.graph_objects as go

# # Charger les données
# geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
# customer = pd.read_csv('data/olist_customers_dataset.csv')
# sellers = pd.read_csv('data/olist_sellers_dataset.csv')

# # Merge des informations géographiques
# geol = pd.merge(customer, geolocation, how='left', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix').groupby('customer_unique_id').mean()[['geolocation_lat', 'geolocation_lng']].reset_index()
# customer_merged = pd.merge(customer, geol, left_on='customer_unique_id', right_on='customer_unique_id')

    
# # # Fusionner les données de géolocalisation avec les données des clients
# # customer_merged = customer.merge(geolocation, on='geolocation_zip_code_prefix', how='left')

# # Fusionner les données de géolocalisation avec les données des vendeurs
# sellers_merged = sellers.merge(geolocation, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

# # Créer une carte interactive pour afficher les vendeurs et les clients par pays
# world_map = go.Figure()

# # Ajouter les marqueurs (vendeurs) à la carte
# world_map.add_trace(
#     go.Scattergeo(
#         lat=sellers_merged['geolocation_lat'],   # Utiliser les latitudes des vendeurs comme emplacements pour les marqueurs
#         lon=sellers_merged['geolocation_lng'],   # Utiliser les longitudes des vendeurs comme emplacements pour les marqueurs
#         mode='markers',
#         marker=dict(size=10, color='blue'),
#         text=sellers_merged['seller_city'],
#         hoverinfo='text'
#     )
# )

# # Ajouter les marqueurs (clients) à la carte
# world_map.add_trace(
#     go.Scattergeo(
#         lat=customer_merged['geolocation_lat'],   # Utiliser les latitudes des clients comme emplacements pour les marqueurs
#         lon=customer_merged['geolocation_lng'],   # Utiliser les longitudes des clients comme emplacements pour les marqueurs
#         mode='markers',
#         marker=dict(size=5, color='red'),
#         text=customer_merged['customer_city'],
#         hoverinfo='text'
#     )
# )

# # Mise en forme de la carte
# world_map.update_layout(
#     title_text='Carte des vendeurs et des clients',
#     title_x=0.5,
#     geo=dict(
#         scope='world',  # Afficher le monde entier sur la carte
#         showland=True,
#         landcolor='rgb(217, 217, 217)',
#         showcountries=True,
#         countrycolor='rgb(255, 255, 255)',
#         showocean=True,
#         oceancolor='rgb(0, 204, 204)',
#         showcoastlines=True,
#         coastlinecolor='rgb(255, 255, 255)',
#         showframe=False,
#         showrivers=False,
#         showlakes=True,
#         lakecolor='rgb(0, 255, 255)'
#     )
# )

# # Affichage de la carte interactive
# world_map.show()


## Carte Client et vendeur (menu déroulant) 

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Charger les données
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
customer = pd.read_csv('data/olist_customers_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')

# Merge des informations géographiques
geol = pd.merge(customer, geolocation, how='left', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix').groupby('customer_unique_id').mean()[['geolocation_lat', 'geolocation_lng']].reset_index()
customer_merged = pd.merge(customer, geol, left_on='customer_unique_id', right_on='customer_unique_id')

# Fusionner les données de géolocalisation avec les données des vendeurs
sellers_merged = sellers.merge(geolocation, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

# Créer une carte interactive pour afficher les vendeurs et les clients par pays
world_map = go.Figure()

# Ajouter les marqueurs (vendeurs) à la carte
vendeurs_trace = go.Scattergeo(
    lat=sellers_merged['geolocation_lat'],   # Utiliser les latitudes des vendeurs comme emplacements pour les marqueurs
    lon=sellers_merged['geolocation_lng'],   # Utiliser les longitudes des vendeurs comme emplacements pour les marqueurs
    mode='markers',
    marker=dict(size=10, color='blue'),
    text=sellers_merged['seller_city'],
    hoverinfo='text',
    name='Vendeurs'
)

# Ajouter les marqueurs (clients) à la carte
clients_trace = go.Scattergeo(
    lat=customer_merged['geolocation_lat'],   # Utiliser les latitudes des clients comme emplacements pour les marqueurs
    lon=customer_merged['geolocation_lng'],   # Utiliser les longitudes des clients comme emplacements pour les marqueurs
    mode='markers',
    marker=dict(size=5, color='red'),
    text=customer_merged['customer_city'],
    hoverinfo='text',
    name='Clients'
)

# Ajouter les traces au layout de la carte
world_map.add_trace(vendeurs_trace)
world_map.add_trace(clients_trace)

# Mise en forme du menu déroulant
world_map.update_layout(
    title_text='Carte des vendeurs et des clients',
    title_x=0.5,
    geo=dict(
        scope='world',  # Afficher le monde entier sur la carte
        showland=True,
        landcolor='rgb(217, 217, 217)',
        showcountries=True,
        countrycolor='rgb(255, 255, 255)',
        showocean=True,
        oceancolor='rgb(0, 204, 204)',
        showcoastlines=True,
        coastlinecolor='rgb(255, 255, 255)',
        showframe=False,
        showrivers=False,
        showlakes=True,
        lakecolor='rgb(0, 255, 255)'
    ),
    # Ajouter un menu déroulant pour afficher tous, vendeurs ou clients
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [{'visible': [True, True]}, {'title': 'Carte des vendeurs et des clients'}],
                    'label': 'Tous',
                    'method': 'update'
                },
                {
                    'args': [{'visible': [True, False]}, {'title': 'Carte des vendeurs'}],
                    'label': 'Vendeurs',
                    'method': 'update'
                },
                {
                    'args': [{'visible': [False, True]}, {'title': 'Carte des clients'}],
                    'label': 'Clients',
                    'method': 'update'
                }
            ],
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ]
)

# Affichage de la carte interactive
world_map.show()


In [ ]:
# Charger les données
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
translation = pd.read_csv('data/product_category_name_translation.csv')

In [ ]:
# Créer une connexion à la base de données DuckDB
con = ddb.connect()

# Requête SQL pour récupérer les localisations des vendeurs sans valeurs nulles ou NaN
query = """
    SELECT
        c.seller_id AS id,
        c.seller_city AS city,
        c.seller_state AS state,
        AVG(g.geolocation_lat) AS lat,
        AVG(g.geolocation_lng) AS lng
    FROM sellers AS c
    LEFT JOIN geolocation AS g ON c.seller_zip_code_prefix = g.geolocation_zip_code_prefix
    WHERE g.geolocation_lat IS NOT NULL AND g.geolocation_lng IS NOT NULL
    GROUP BY c.seller_id, c.seller_city, c.seller_state
"""

# Exécuter la requête
locations_df = con.execute(query).fetchdf()
locations_df.sample(5)
# # Créer une carte
# m = folium.Map(location=[locations_df['lat'].mean(), locations_df['lng'].mean()], zoom_start=4)

# # Ajouter des marqueurs à la carte
# marker_cluster = MarkerCluster().add_to(m)
# for idx, row in locations_df.iterrows():
#     folium.Marker(
#         location=[row['lat'], row['lng']],
#         popup=row['city'] + ', ' + row['state']
#     ).add_to(marker_cluster)

# # Afficher le menu déroulant
# folium.LayerControl().add_to(m)

# # Afficher la carte interactive
# m


In [ ]:
# Création du GeoDataFrame à partir du DataFrame geolocalisation
gdf = gpd.GeoDataFrame(
    locations_df, 
    geometry=gpd.points_from_xy(locations_df['lng'], locations_df['lat'])
)

# Création de la carte centrée sur la position moyenne des points
ax = gdf.plot(figsize=(10, 10), alpha=0.5, markersize=10)

# Affichage de la carte
plt.show()

In [ ]:
# Importer la bibliothèque Plotly pour créer une carte interactive
import plotly.graph_objects as go

# Création de la carte centrée sur la position moyenne des points
m = go.Figure(go.Scattermapbox(
    lat=locations_df['lat'],      # Coordonnées de latitude des points
    lon=locations_df['lng'],      # Coordonnées de longitude des points
    mode='markers',                           # Mode d'affichage des marqueurs (ici, des points)
    marker=go.scattermapbox.Marker(size=10),  # Taille des marqueurs (points) sur la carte
    text=locations_df['city'],       # Texte affiché lorsqu'on survole un point (ville du client)
))

# Mise à jour du style de la carte avec une carte de fond "open-street-map"
# Définir une variable pour stocker le niveau de zoom souhaité
zoom_level = 2

m.update_layout(
    mapbox_style='open-street-map',
    
    # Centrage de la carte sur la position moyenne des points (latitude et longitude)
    mapbox_center={'lat': locations_df['lat'].mean(), 'lon': locations_df['lng'].mean()},
    
    # Utiliser la variable 'zoom_level' pour définir le niveau de zoom initial de la carte
    mapbox_zoom=zoom_level,
)

# Affichage de la carte interactive
m.show()

In [ ]:
# Création de la carte
m = folium.Map(location=[locations_df['lat'].mean(), locations_df['lng'].mean()], zoom_start=4)

# Ajout des clusters de marqueurs
marker_cluster = MarkerCluster().add_to(m)
for idx, row in locations_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['city'] + ', ' + row['state']
    ).add_to(marker_cluster)

# Afficher le menu déroulant
folium.LayerControl().add_to(m)

# Afficher la carte interactive
m

In [ ]:
# Créer une connexion à la base de données DuckDB
con = ddb.connect()

# # Requête SQL pour récupérer les localisations uniques des clients sans valeurs nulles ou NaN
# query = """
#     SELECT DISTINCT
#         c.customer_id AS id,
#         c.customer_city AS city,
#         c.customer_state AS state,
#         g.geolocation_lat AS lat,
#         g.geolocation_lng AS lng
#     FROM customers AS c
#     LEFT JOIN geolocation AS g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
#     WHERE g.geolocation_lat IS NOT NULL AND g.geolocation_lng IS NOT NULL
# """
# Requête SQL pour récupérer les localisations uniques des clients sans valeurs nulles ou NaN
query = """
    SELECT
        c.customer_id AS id,
        c.customer_city AS city,
        c.customer_state AS state,
        AVG(g.geolocation_lat) AS lat,
        AVG(g.geolocation_lng) AS lng
    FROM customers AS c
    LEFT JOIN geolocation AS g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
    WHERE g.geolocation_lat IS NOT NULL AND g.geolocation_lng IS NOT NULL
    GROUP BY c.customer_id, c.customer_city, c.customer_state
"""

# Exécuter la requête
locations_df = con.execute(query).fetchdf()
locations_df.sample(5)


In [ ]:
# Création du GeoDataFrame à partir du DataFrame geolocalisation
gdf = gpd.GeoDataFrame(
    locations_df, 
    geometry=gpd.points_from_xy(locations_df['lng'], locations_df['lat'])
)

# Création de la carte centrée sur la position moyenne des points
ax = gdf.plot(figsize=(10, 10), alpha=0.5, markersize=10)

# Affichage de la carte #
plt.show()

In [ ]:
# import folium

# # Création de la carte centrée sur la position moyenne des points
# m = folium.Map(location=[locations_df['lat'].mean(), locations_df['lng'].mean()], zoom_start=10)

# # Ajout des marqueurs pour chaque point
# for idx, row in locations_df.iterrows():
#     folium.Marker([row['lat'], row['lng']]).add_to(m)

# # Affichage de la carte
# m.save('map.html')  # Sauvegarde de la carte dans un fichier HTML


In [ ]:
# Importer la bibliothèque Plotly pour créer une carte interactive
import plotly.graph_objects as go

# Création de la carte centrée sur la position moyenne des points
m = go.Figure(go.Scattermapbox(
    lat=locations_df['lat'],      # Coordonnées de latitude des points
    lon=locations_df['lng'],      # Coordonnées de longitude des points
    mode='markers',                           # Mode d'affichage des marqueurs (ici, des points)
    marker=go.scattermapbox.Marker(size=10),  # Taille des marqueurs (points) sur la carte
    text=locations_df['city'],       # Texte affiché lorsqu'on survole un point (ville du client)
))

# Mise à jour du style de la carte avec une carte de fond "open-street-map"
# Définir une variable pour stocker le niveau de zoom souhaité
zoom_level = 2

m.update_layout(
    mapbox_style='open-street-map',
    
    # Centrage de la carte sur la position moyenne des points (latitude et longitude)
    mapbox_center={'lat': locations_df['lat'].mean(), 'lon': locations_df['lng'].mean()},
    
    # Utiliser la variable 'zoom_level' pour définir le niveau de zoom initial de la carte
    mapbox_zoom=zoom_level,
)

# Affichage de la carte interactive
m.show()

In [ ]:
# Création de la carte
m = folium.Map(location=[locations_df['lat'].mean(), locations_df['lng'].mean()], zoom_start=4)

# Ajout des clusters de marqueurs
marker_cluster = MarkerCluster().add_to(m)
for idx, row in locations_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['city'] + ', ' + row['state']
    ).add_to(marker_cluster)

# Afficher le menu déroulant
folium.LayerControl().add_to(m)

# Afficher la carte interactive
m

### Distribution des commandes par mois

In [ ]:
orders["period"] = orders[["order_purchase_timestamp"]].applymap(lambda o: str(dt.datetime.strptime(
    o, '%Y-%m-%d %H:%M:%S').year * 100 + dt.datetime.strptime(o, '%Y-%m-%d %H:%M:%S').month))
sorted_orders = orders.sort_values(by=["period"])

fig = plt.figure(figsize=[24, 8])
sns.displot(sorted_orders["period"])
plt.xticks(rotation=90)
plt.show()


In [ ]:
import datetime as dt

rfm_df = ddb.query(
    """
    SELECT
        customer_unique_id, 
        MAX(order_purchase_timestamp) as last_order,
        COUNT(DISTINCT o.order_id) as frequency,
        SUM(payment_value) as monetary_value
    FROM customers as c
    JOIN orders as o on c.customer_id = o.customer_id
    JOIN order_payments as p on o.order_id = p.order_id
    WHERE order_purchase_timestamp <= '2018-08-31 23:59:59'
    AND   order_purchase_timestamp >= '2017-09-01 00:00:00'
    GROUP BY customer_unique_id
    HAVING SUM(payment_value) <= 5500
    """
).to_df()

rfm_df["recency"] = rfm_df[["last_order"]].applymap(lambda o: (dt.datetime.strptime('2018-08-31 23:59:59', '%Y-%m-%d %H:%M:%S') - dt.datetime.strptime(o, '%Y-%m-%d %H:%M:%S')).days)
rfm_df["mean_monetary_value"] = rfm_df.monetary_value / rfm_df.frequency

# Calculer la date de la dernière commande (récence)
# current_date = df['order_purchase_timestamp'].max()
# df['recency'] = (current_date - df['order_purchase_timestamp']).dt.days

rfm_df.head()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Création d'un graphique en 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Ajout des points sur le graphique en 3D
# ax.scatter(rfm_df['recency'], rfm_df['frequency'], rfm_df['monetary_value'], c=rfm_df['monetary_value'], cmap='viridis', alpha=0.8)
ax.scatter(rfm_df['recency'], rfm_df['frequency'], rfm_df['mean_monetary_value'], c=rfm_df['mean_monetary_value'], cmap='viridis', alpha=0.8)

# Configuration des axes
ax.set_xlabel('Récence (jours)')
ax.set_ylabel('Fréquence')
ax.set_zlabel('Valeur monétaire')
ax.set_title('Segmentation RFM')

# Ajout de la barre de couleur pour représenter la valeur monétaire
cbar = plt.colorbar(ax.scatter([], [], [], c=[], cmap='viridis'), ax=ax)
cbar.set_label('Valeur monétaire')

plt.show()


In [ ]:
sns.scatterplot(x=rfm_df.recency, y=rfm_df.monetary_value)
plt.show()
sns.scatterplot(x=rfm_df.recency, y=rfm_df.frequency)
plt.show()
sns.scatterplot(x=rfm_df.frequency, y=rfm_df.recency)
plt.show()
sns.scatterplot(x=rfm_df.frequency, y=rfm_df.monetary_value)
plt.show()

In [ ]:
sns.scatterplot(x=rfm_df.recency, y=rfm_df.mean_monetary_value)
plt.show()
sns.scatterplot(x=rfm_df.recency, y=rfm_df.frequency)
plt.show()
sns.scatterplot(x=rfm_df.frequency, y=rfm_df.recency)
plt.show()
sns.scatterplot(x=rfm_df.frequency, y=rfm_df.mean_monetary_value)
plt.show()

| v Commandes - récence -> | Low | Middle | Top |
| --- | --- | --- | --- |
| Low |  |  |  |
| Middle |  | Main group | A relancer |
| Top |  |  | Gold Customer |


In [ ]:
# Enregistrer le DataFrame dans un fichier CSV
rfm_df.to_csv('data/data_cleaned_rfm.csv', index=False)